In [17]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from camera_matrix import camera_matrix
import time
import seaborn as sns
import matplotlib.patches as mpatches
import math

In [15]:
focal_length = 27 # in mm
# Diagonal size of the sensor (in mm)
diagonal_size = 6.17

# Aspect ratio of the sensor
aspect_ratio = 4 / 3

# Calculate the sensor height
sensor_height = math.sqrt((diagonal_size**2) / (1 + aspect_ratio**2))

# Calculate the sensor width
sensor_width = aspect_ratio * sensor_height

print("Sensor Width of GoPro HERO 10 Black:", sensor_width, "mm")


# Compute depth
def compute_depth(matches, keypoints1, keypoints2):
    depth_list = []
    for match in matches:
        pt1 = keypoints1[match.queryIdx].pt
        pt2 = keypoints2[match.trainIdx].pt
        disparity = abs(pt1[0] - pt2[0])
        depth = (focal_length * disparity) / sensor_width
        depth_list.append(depth)
    return depth_list

Sensor Width of GoPro HERO 10 Black: 4.936 mm


In [19]:
cam1 = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_dx.MP4")
ret, image1 = cam1.read()
cam2= cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_sx.MP4")
ret, image2 = cam2.read()

# Calibration parameters (obtained from camera calibration)
camera_matrix = np.array([[focal_length, 0, image1.shape[1]/2],
                          [0, focal_length, image1.shape[0]/2],
                          [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((4,1))

# Triangulation function
def triangulate_points(matches, keypoints1, keypoints2):
    points_3d = []
    for match in matches:
        pt1 = keypoints1[match.queryIdx].pt
        pt2 = keypoints2[match.trainIdx].pt
        point_3d = cv2.triangulatePoints(np.eye(3,4), np.eye(3,4), pt1, pt2)
        point_3d /= point_3d[3]
        points_3d.append(point_3d[:3])
    return np.array(points_3d)

# Initialize feature detector (SIFT, ORB, etc.)
detector = cv2.SIFT_create()

# Find keypoints and descriptors
keypoints1, descriptors1 = detector.detectAndCompute(image1, None)
keypoints2, descriptors2 = detector.detectAndCompute(image2, None)

# Initialize matcher
matcher = cv2.BFMatcher()

# Match descriptors
matches = matcher.match(descriptors1, descriptors2)

# Sort matches by score
matches = sorted(matches, key=lambda x: x.distance)
matches = matches[:15]

# Triangulate points
points_3d = triangulate_points(matches, keypoints1, keypoints2)

# Print 3D points
for i, point in enumerate(points_3d):
    print("Point {}: Coordinates = {}".format(i+1, point))

# Draw top matches
matched_img = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display result
cv2.imshow('Matches', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Point 1: Coordinates = [[0.]
 [0.]
 [0.]]
Point 2: Coordinates = [[0.]
 [0.]
 [0.]]
Point 3: Coordinates = [[0.]
 [0.]
 [0.]]
Point 4: Coordinates = [[0.]
 [0.]
 [0.]]
Point 5: Coordinates = [[0.]
 [0.]
 [0.]]
Point 6: Coordinates = [[0.]
 [0.]
 [0.]]
Point 7: Coordinates = [[0.]
 [0.]
 [0.]]
Point 8: Coordinates = [[0.]
 [0.]
 [0.]]
Point 9: Coordinates = [[0.]
 [0.]
 [0.]]
Point 10: Coordinates = [[0.]
 [0.]
 [0.]]
Point 11: Coordinates = [[0.]
 [0.]
 [0.]]
Point 12: Coordinates = [[0.]
 [0.]
 [0.]]
Point 13: Coordinates = [[0.]
 [0.]
 [0.]]
Point 14: Coordinates = [[0.]
 [0.]
 [0.]]
Point 15: Coordinates = [[0.]
 [0.]
 [0.]]


In [16]:
import numpy as np
import cv2

cam1 = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_dx.MP4")
ret, image1 = cam1.read()
cam2= cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_sx.MP4")
ret, image2 = cam2.read()

image_width = image1.shape[1] # in pixels

# Initialize feature detector (SIFT, ORB, etc.)
detector = cv2.SIFT_create()

# Find keypoints and descriptors
keypoints1, descriptors1 = detector.detectAndCompute(image1, None)
keypoints2, descriptors2 = detector.detectAndCompute(image2, None)

# Initialize matcher
matcher = cv2.BFMatcher()

# Match descriptors
matches = matcher.match(descriptors1, descriptors2)

# Sort matches by score
matches = sorted(matches, key=lambda x: x.distance)
matches = matches[:15]
# Compute depth
depth_list = compute_depth(matches, keypoints1, keypoints2)

# Print depth for each match
for i, match in enumerate(matches):
    print("Match {}: Depth = {} mm".format(i+1, depth_list[i]))

# Draw top matches
matched_img = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display result
cv2.imshow('Matches', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Match 1: Depth = 964.4722961722549 mm
Match 2: Depth = 993.7681324864516 mm
Match 3: Depth = 964.3107063380014 mm
Match 4: Depth = 1100.04750256407 mm
Match 5: Depth = 1084.7809357233527 mm
Match 6: Depth = 1169.8356146170988 mm
Match 7: Depth = 1102.1548476752432 mm
Match 8: Depth = 1063.3051793424586 mm
Match 9: Depth = 987.8988157794773 mm
Match 10: Depth = 1121.4454687974828 mm
Match 11: Depth = 1083.5319531408277 mm
Match 12: Depth = 993.7681324864516 mm
Match 13: Depth = 986.4712163347093 mm
Match 14: Depth = 2884.9501153831543 mm
Match 15: Depth = 1279.1945397177624 mm
